In [119]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.pagina12.com.ar/'
p12 = requests.get(url)
s = BeautifulSoup(p12.text,'lxml')

In [14]:
secciones = s.find('ul',attrs={'class':'horizontal-list main-sections hide-on-dropdown'}).find_all('li')
link_secciones = [secion.a.get('href') for secion in secciones]

In [64]:
link_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/el-mundo',
 'https://www.pagina12.com.ar/secciones/cultura']

In [98]:
box_secciones = [requests.get(seccion) for seccion in link_secciones]
box_links =[BeautifulSoup(i.text,'lxml') for i in box_secciones]
box_link =[i.find_all('h4',attrs={'class':'is-display-inline title-list'}) for i in box_links]

In [114]:
all_links = []
for i in box_link:
    if i is not None:
        for j in i:
            all_links.append(j.a.get('href'))

In [115]:
all_links

['https://www.pagina12.com.ar/319202-el-gobierno-habilito-al-congreso-a-tratar-el-proyecto-para-s',
 'https://www.pagina12.com.ar/319140-vuelta-a-clases-en-caba-los-docentes-advierten-que-haran-un-',
 'https://www.pagina12.com.ar/319141-una-ayuda-para-las-obras-sociales',
 'https://www.pagina12.com.ar/319143-coronavirus-axel-kicillof-anuncio-que-la-semana-proxima-comi',
 'https://www.pagina12.com.ar/319151-los-movimientos-sociales-le-ponen-el-hombro-a-la-campana-de-',
 'https://www.pagina12.com.ar/319156-la-venia-envenenada-y-el-silencio-lelo',
 'https://www.pagina12.com.ar/319157-la-interna-de-juntos-por-el-cambio-dardos-de-miguel-pichetto',
 'https://www.pagina12.com.ar/319159-las-dietas-de-los-legisladores-seguiran-congeladas-seis-mese',
 'https://www.pagina12.com.ar/319210-ahora-es-el-turno-del-fmi',
 'https://www.pagina12.com.ar/319199-es-oficial-la-prorroga-de-la-prohibicion-de-despidos-por-90-',
 'https://www.pagina12.com.ar/319049-subiran-el-monto-de-la-tarjeta-alimentar',
 'ht

In [88]:
link

[<h4 class="is-display-inline title-list"><a class="" href="https://www.pagina12.com.ar/317616-magnus-carlsen-un-comentarista-de-lujo-para-gambito-de-dama">Magnus Carlsen, un comentarista de lujo para "Gambito de dama"</a></h4>,
 <h4 class="is-display-inline title-list"><a class="" href="https://www.pagina12.com.ar/317154-francia-se-vuelve-a-fanatizar-con-arsene-lupin">Francia se vuelve a fanatizar con Arséne Lupin </a></h4>,
 <h4 class="is-display-inline title-list"><a class="" href="https://www.pagina12.com.ar/317112-shakira-vendio-los-derechos-de-un-centenar-de-canciones-a-un">Shakira vendió los derechos de un centenar de canciones a un fondo de inversión </a></h4>,
 <h4 class="is-display-inline title-list"><a class="" href="https://www.pagina12.com.ar/316288-el-instituto-de-carreras-alternativas-de-leo-masliah">El instituto de carreras alternativas de Leo Maslíah</a></h4>,
 <h4 class="is-display-inline title-list"><a class="" href="https://www.pagina12.com.ar/316130-murio-la-sargen

In [58]:
def obtener_info(soup):
    dic_nota = {}
    

    #fecha
        
    fecha_article = soup.find('span',attrs={'pubdate':'pubdate'})
    if fecha_article:
        dic_nota['fecha'] = fecha_article.get('datetime')
        
        #Volanta 
        
    volanta_article = soup.find('h2',attrs={'class':'article-prefix'})
    if volanta_article:
        dic_nota['volanta'] = volanta_article.text
    else:
        dic_nota['volanta'] = None
        
        #Copete
        
    copete_article = soup.find('div',attrs={'class':'article-summary'})
    if copete_article:
        dic_nota['copete'] = copete_article.text
    else:
        dic_nota['copete'] = None
            
        #Imagen
        
    imagen_article = soup.find('div',attrs={'class':'article-main-media-image'})
    if imagen_article:
        imagenes = imagen_article.find_all('img')
            
        if len(imagenes) != 0:
            imagen_src = imagenes[-1].get('data-src')  
                
            try:
                imagen_req = requests.get(imagen_src)
                    
                if imagen_req.status_code == 200:
                    dic_nota['imagen'] = imagen_req.content
                else:
                    print('La imagen no esta disponible')
                
            except Exception as e:
                    print('Error al hacer el request {e}')         
        else:
            print('Error no hay imagenes')
        #contenido
        
    article_text = soup.find('div',attrs={'class':"article-text"})
    if article_text:
            dic_nota['contenido'] = article_text.get_text()
        
    else:
        dic_nota['contenido'] = None
        
    return dic_nota














    #try:
        
        
    
    #except Exception as e:
        #print(f' -*-  Error  -*-\n {e}\n')
        
        

In [59]:
def scrape_nota(url):
    try:
        nota = requests.get(url)
    except Exception as e :
        print('\nError')
        print(e)
        return None
    
    if nota.status_code == 200:
        nota_s = BeautifulSoup(nota.text,'lxml')
        dic_nota = obtener_info(nota_s)
        dic_nota['url'] = url
        
        
    else:
        print(f'Error en la {url}')
        print('El status_code no es 200')
        return None
        

    
    return dic_nota
    

In [117]:
data = []

for idx , nota in enumerate(all_links):
    print(f'Escrapenado link {idx}/{len(all_links)-1}')
    data.append(scrape_nota(nota))
    
    

Escrapenado link 0/47
Escrapenado link 1/47
Escrapenado link 2/47
Escrapenado link 3/47
Escrapenado link 4/47
Escrapenado link 5/47
Escrapenado link 6/47
Escrapenado link 7/47
Escrapenado link 8/47
Escrapenado link 9/47
Escrapenado link 10/47
Escrapenado link 11/47
Escrapenado link 12/47
Escrapenado link 13/47
Escrapenado link 14/47
Escrapenado link 15/47
Escrapenado link 16/47
Escrapenado link 17/47
Escrapenado link 18/47
Escrapenado link 19/47
Escrapenado link 20/47
Escrapenado link 21/47
Escrapenado link 22/47
Escrapenado link 23/47
Escrapenado link 24/47
Escrapenado link 25/47
Escrapenado link 26/47
Escrapenado link 27/47
Escrapenado link 28/47
Escrapenado link 29/47
Escrapenado link 30/47
Escrapenado link 31/47
Escrapenado link 32/47
Escrapenado link 33/47
Escrapenado link 34/47
Escrapenado link 35/47
Escrapenado link 36/47
Escrapenado link 37/47
Escrapenado link 38/47
Escrapenado link 39/47
Escrapenado link 40/47
Escrapenado link 41/47
Escrapenado link 42/47
Escrapenado link 43/4

In [120]:
data_frama = pd.DataFrame(data)

In [122]:
data_frama.head()

,fecha,volanta,copete,imagen,contenido,url
0,2021-01-23,Con un decreto firmado este viernes,La iniciativa es impulsada por algunos goberna...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,El Gobierno amplió con un decreto firmado este...,https://www.pagina12.com.ar/319202-el-gobierno...
1,2021-01-23,El gobierno porteño amenaza con descontarles l...,Los maestros plantearon que no hay condiciones...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Después del anuncio por parte del jefe del gob...,https://www.pagina12.com.ar/319140-vuelta-a-cl...
2,2021-01-23,El Gobierno dará un auxilio en el marco de la ...,La medida alcanzará a 46 sindicatos y se repar...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"El Gobierno otorgó 227,7 millones de pesos a...",https://www.pagina12.com.ar/319141-una-ayuda-p...
3,2021-01-23,Los preparativos para la vuelta a clases en la...,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"El gobernador de la provincia de Buenos Aires,...",https://www.pagina12.com.ar/319143-coronavirus...
4,2021-01-23,Promueven la aplicación de la vacuna en los ba...,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Los movimientos sociales comenzaron a trabajar...,https://www.pagina12.com.ar/319151-los-movimie...


In [123]:
data_frama.to_csv('notas_p12.csv')